In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
links_data = pd.read_csv('houses_link.csv')
list_urls = links_data['house link'].tolist()
print(list_urls[:2])

['/avanti/el-retiro/proyecto-nuevo-det-4283005.aspx', '/apartamento-en-venta/el-retiro/retiro-det-4705196.aspx']


In [3]:
main_url = 'https://www.fincaraiz.com.co'
list_urls = [f'{main_url}{url}' for url in list_urls]
list_urls[:2]

['https://www.fincaraiz.com.co/avanti/el-retiro/proyecto-nuevo-det-4283005.aspx',
 'https://www.fincaraiz.com.co/apartamento-en-venta/el-retiro/retiro-det-4705196.aspx']

In [4]:
def get_title_and_price(soup):
    try:
        detail = soup.find('div', attrs={'class':'detailheader'})
    
        title = detail.find('div', attrs={'class', 'title'})
        title_box = title.find('div', attrs={'class', 'box'})
        title_text = title_box.find('h1').text
    
        price = detail.find('div', attrs={'class', 'price'})
        price_text = price.find('h2').text
        
        return title_text, price_text
    except Exception as e:
        pass

In [5]:
def get_house_features(soup):
    try:
        features = soup.find('div', attrs={'class': 'features'})
        surface = features.find('span', attrs={'class': 'advertSurface'}).text.strip()
        rooms = features.find('span', attrs={'class': 'advertRooms'}).text.strip()
        baths = features.find('span', attrs={'class': 'advertBaths'}).text.strip()
        garages = features.find('span', attrs={'class': 'advertGarages'}).text.strip()
        
        return clean_features_text(surface, rooms, baths, garages)        
    except Exception as e:
        pass

In [6]:
def get_house_features_2(soup):
    try:
        features_2 = soup.find('div', attrs={'class': 'features_2'})
        features = features_2.find_all('li')
        features = features[:-1]
    
        features_2 = dict()
        for feature in features:
            feature_key = feature.find('b').text
            feature_value = feature.text
            features_2.update({feature_key: feature_value})
        
        return clean_features_2(features_2)
    except Exception as e:
        pass

In [7]:
def clean_features_text(surface, rooms, baths, garages):
    rooms = rooms[-1:]
    baths = baths[-1:]
    garages = garages[-1:] if garages != 'Sin especificar' else 'nan'
    
    return surface, rooms, baths, garages

In [8]:
def clean_features_2(features):
    private_area = features.get('Área privada:')
    builded_area = features.get('Área Const.:')
    estrato = features.get('Estrato:')
    antiquity = features.get('Antigüedad:')
    price_by_m = features.get('Precio m²:')
    administration = features.get('Admón:')
    
    private_area = delete_dirty_chars(private_area).split(':')[1] if private_area else 'nan'
    builded_area = delete_dirty_chars(builded_area).split(':')[1] if builded_area else 'nan'
    estrato = delete_dirty_chars(estrato).split(':')[1][0] if estrato else 'nan'
    antiquity = delete_dirty_chars(antiquity).split(':')[1] if antiquity else 'nan'
    price_by_m = delete_dirty_chars(price_by_m).split(':')[1] if price_by_m else 'nan'
    administration = delete_dirty_chars(administration).split(':')[1][0:8] if administration else 'nan'
    
    return (private_area, builded_area, estrato, antiquity, price_by_m, administration)    

In [9]:
def delete_dirty_chars(string):
    return string.replace(' ', '').replace('\n', '').replace('\r', '')

In [10]:
all_data = []
for index, url in enumerate(list_urls):
    try:
        r = requests.get(url)
        if r.status_code == 404:
            continue
        soup = BeautifulSoup(r.text, 'html.parser')
    
        headers = get_title_and_price(soup)
        features = get_house_features(soup)
        features_2 = get_house_features_2(soup)
    
        house = headers + features + features_2
        all_data.append(house)
    
    except Exception as e:
        continue
    
    print('---------------------------------------------------------')
    print(f'{index}, { url}')
    print('---------------------------------------------------------')
    print(house)
    print('---------------------------------------------------------')

---------------------------------------------------------
0, https://www.fincaraiz.com.co/avanti/el-retiro/proyecto-nuevo-det-4283005.aspx
---------------------------------------------------------
('AVANTI', 'Desde $ 239.687.000 Hasta $ 405.928.824', '47,50 a 79,57 m²', '1', '1', '1', '46,34a76,76m²', '47,50a79,57m²', '4', 'nan', 'nan', 'nan')
---------------------------------------------------------
---------------------------------------------------------
1, https://www.fincaraiz.com.co/apartamento-en-venta/el-retiro/retiro-det-4705196.aspx
---------------------------------------------------------
('Apartamento en Venta El Retiro EL Retiro', '$ 255.000.000', '60,00 m²', '2', '2', '1', 'nan', '60,00m²', '4', 'Menosde1año', '4.250.000/m²', '$120,000')
---------------------------------------------------------
---------------------------------------------------------
2, https://www.fincaraiz.com.co/apartamento-en-venta/el-retiro/bosques_normandia-det-4635130.aspx
------------------------

In [14]:
columns = ['title', 'price', 'surface', 'rooms', 'baths', 'garages', 'private area', 
           'builded area', 'estrato', 'antiquity', 'price m2', 'administration']
data_frame = pd.DataFrame(all_data, columns=columns)

In [15]:
data_frame.to_csv('retiro_houses.csv', index=False, encoding='utf-8')